In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler, label_binarize
import warnings
import os
warnings.filterwarnings('ignore')

In [ ]:
# Set global font settings
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# Font size configuration
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 26

print("Font settings configured")

## Data Loading

In [ ]:
data_file = r"F:\作图目录20280825\骨质疏松数据.xlsx"
df = pd.read_excel(data_file, header=1, usecols='B:H') 
cols = ['signal_1', 'sost_1', 'signal_2', 'sost_2', 'sost_mean', 'l1_4', 'left_hip']
df.columns = cols

print(f"Data dimensions: {df.shape}")
print(f"Features: {list(df.columns)}")

In [ ]:
# Add class labels
if len(df) == 103:
    df['class'] = ['Healthy'] * 35 + ['Osteopenia'] * 33 + ['Osteoporosis'] * 35
else:
    df['class'] = ['Healthy'] * 35 + ['Osteopenia'] * 33 + ['Osteoporosis'] * 36

print("Class distribution:")
print(df['class'].value_counts())

y = df['class'].map({'Healthy': 0, 'Osteopenia': 1, 'Osteoporosis': 2})
print(f"\nLabel mapping: {dict(zip(['Healthy', 'Osteopenia', 'Osteoporosis'], [0, 1, 2]))}")

## SVM Configuration

In [ ]:
svm_config = {
    'model': SVC(probability=True, random_state=42),
    'params': {'C': [0.1, 1, 10], 'kernel': ['rbf', 'linear']},
    'color': '#24AAE3'
}

## Core Functions

In [ ]:
def train_svm(config, X_train, X_test, y_train, y_test):
    """Train SVM algorithm and return prediction probabilities"""
    print("Training: SVM...")
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    grid_search = GridSearchCV(config['model'], config['params'], cv=3, scoring='accuracy')
    grid_search.fit(X_train_scaled, y_train)
    best_model = grid_search.best_estimator_
    print(f"  ✓ Best parameters: {grid_search.best_params_}")
    
    y_proba = best_model.predict_proba(X_test_scaled)
    return y_proba

In [ ]:
def plot_svm_roc_curve(y_true, y_proba, title, save_path=None):
    """Plot SVM ROC curve"""
    
    y_bin = label_binarize(y_true, classes=[0, 1, 2])
    n_classes = y_bin.shape[1]
    
    fig, ax = plt.subplots(figsize=(14, 11))
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], y_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Calculate macro-average ROC
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    # Plot macro-average ROC curve
    ax.plot(fpr["macro"], tpr["macro"],
            label=f'SVM (AUC = {roc_auc["macro"]:.3f})',
            color=svm_config['color'], linestyle='-', linewidth=4, alpha=0.9)
    
    # Diagonal reference line
    ax.plot([0, 1], [0, 1], 'k--', linewidth=2.5, alpha=0.8, label='Random Classifier')
    
    ax.set_xlim([-0.02, 1.02])
    ax.set_ylim([-0.02, 1.02])
    
    ax.set_xlabel('False Positive Rate', fontsize=22, fontweight='bold', labelpad=15)
    ax.set_ylabel('True Positive Rate', fontsize=22, fontweight='bold', labelpad=15)
    ax.set_title(title, fontsize=26, fontweight='bold', pad=30)
    
    ax.set_xticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    ax.tick_params(axis='both', which='major', labelsize=18, length=8, width=2)
    
    legend = ax.legend(loc="lower right", fontsize=16, frameon=True, 
                       fancybox=True, shadow=True, framealpha=0.95,
                       edgecolor='black', facecolor='white',
                       borderpad=1.2, labelspacing=1.5)
    legend.get_frame().set_linewidth(2)
    
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')
    
    for spine in ax.spines.values():
        spine.set_linewidth(3)
        spine.set_color('black')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(f"{save_path}_high_res.tiff", dpi=600, bbox_inches='tight', 
                    facecolor='white', format='tiff')
        plt.savefig(f"{save_path}_vector.pdf", format='pdf', bbox_inches='tight', 
                    facecolor='white')
        print(f"✓ Saved: {save_path}_high_res.tiff and {save_path}_vector.pdf")
    
    plt.show()
    return roc_auc["macro"]

## Generate SOST1 + SOST2 SVM ROC Curve

In [ ]:
print("=== Generating SOST1 + SOST2 SVM ROC curve ===")

X_sost = df[['sost_1', 'sost_2']].values
X_sost_train, X_sost_test, y_sost_train, y_sost_test = train_test_split(
    X_sost, y, test_size=0.25, random_state=42, stratify=y)

y_proba_svm = train_svm(svm_config, X_sost_train, X_sost_test, 
                        y_sost_train, y_sost_test)

output_dir = r"F:\作图目录20280825\ROC_curves_enhanced"
os.makedirs(output_dir, exist_ok=True)

save_path_svm = os.path.join(output_dir, "ROC_SOST1_SOST2_SVM_only")
auc_score = plot_svm_roc_curve(
    y_sost_test, y_proba_svm, 
    'SOST1 + SOST2 Features - SVM', 
    save_path_svm
)

print(f"\n=== SVM AUC Score: {auc_score:.3f} ===")
print("\n=== SOST1+SOST2 SVM completed ===")

## Results Summary

In [ ]:
print("=== SVM Performance Summary ===")
print(f"Feature Combination: SOST1+SOST2")
print(f"Algorithm: SVM")
print(f"AUC Score: {auc_score:.3f}")
print(f"Color: {svm_config['color']}")

result_data = {
    'Feature Combination': ['SOST1+SOST2'],
    'Algorithm': ['SVM'],
    'AUC Score': [f"{auc_score:.3f}"],
    'Color': [svm_config['color']]
}

result_df = pd.DataFrame(result_data)
summary_file = os.path.join(output_dir, "SVM_Performance_Summary.xlsx")
result_df.to_excel(summary_file, index=False)
print(f"\n✓ Results saved to: {summary_file}")